In [1]:
!pip install sentence_transformers

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import os
import csv
import math
import torch
import logging
import pandas as pd
from datetime import datetime
from torch.utils.data import DataLoader, Dataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample

from sentence_transformers import SentenceTransformer,util
import torch
import scipy
from scipy.stats import pearsonr

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    logging.info(f'Using GPU: {torch.cuda.get_device_name()}')
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    logging.info('Using CPU')
    print('Using CPU')

Using CPU


In [3]:
PATH="data/Task1/"

In [4]:


class TSDataset(Dataset):
    def __init__(self, file_path):
        data = []
        with open(file_path, encoding="utf8") as f:
            reader = csv.DictReader(f, delimiter="\t")
            for row in reader:
                data.append(InputExample(texts=[row["sentence1"], row["sentence2"]], label=(float(row["score"]))/5))

        self.samples = data

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        return sample


In [5]:
model_name = "all-MiniLM-L6-v2"
train_batch_size = 32
num_epochs = 4

In [6]:
logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()]
)

In [7]:
model_save_path = (
    "output/training-" + model_name 
)


In [8]:
model = SentenceTransformer(model_name)
model.to(device)
logging.info("Read train dataset")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rahul\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
train_dataset = TSDataset(f"{PATH}train.csv")
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


In [10]:
logging.info("Read dev dataset")
dev_dataset = TSDataset(f"{PATH}dev.csv")

# dev_input_examples = [InputExample(texts=[sentence1, sentence2], label=score) for sentence1, sentence2, score in dev_dataset]

# evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_input_examples, name="dev")
# evaluator = EmbeddingSimilarityEvaluator(dev_dataset)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_dataset, name="dev")


In [11]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

In [12]:
epochs = 4
for epoch in range(epochs):
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=evaluator,
        evaluation_steps=500,
        warmup_steps=warmup_steps,
        output_path=model_save_path,
        # device=device
    )
    
    torch.save(model.state_dict(), f"output/model1C_epoch_{epoch + 1}.pth")

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/179 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# test_dataset = TSDataset(f"{PATH}test.csv")

In [ ]:
untrained_model = SentenceTransformer(model_name)
untrained_model.to(device)

loaded_model = SentenceTransformer(model_save_path)
loaded_model.to(device)

models=[untrained_model, loaded_model]

In [ ]:
validation_data = pd.read_csv(f"{PATH}dev.csv",sep="\t")
validation_data
validation_data.dropna(inplace=True)

In [ ]:
sentence_set_1 = validation_data['sentence1'].to_numpy()
sentence_set_2 = validation_data['sentence2'].to_numpy()
sentence_set_1.shape,sentence_set_2.shape

((1468,), (1468,))

In [ ]:
for model in models:
    print(f"Model: {model}")
    encode_sentence_set_1 = model.encode(sentence_set_1)
    encode_sentence_set_2 = model.encode(sentence_set_2)

    cosine_similarities = util.cos_sim(encode_sentence_set_1,encode_sentence_set_2).tolist()

    predicted_scores = []
    pairwise_pearson=[]
    for i in range(len(cosine_similarities)):
        cos_score = (cosine_similarities[i][i]+1)/2
        pairwise_pearson.append(pearsonr(encode_sentence_set_1[i],encode_sentence_set_2[i])[0])
        predicted_scores.append(cos_score*5)
        
        
    data={
        'pairwise_pearson':pairwise_pearson,
        'predicted_score':predicted_scores,
        'sentence1':sentence_set_1,
        'sentence2':sentence_set_2
    }
    final_dataframe = pd.DataFrame(data)
    final_dataframe.to_csv("task_1_B.csv")
    final_dataframe

    score_val = validation_data['score']
    score_pred_val = final_dataframe['predicted_score']

    print(f"Pearson's Coeffecient = {pearsonr(score_val,score_pred_val)[0]}")

Model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Pearson's Coeffecient = 0.8631423812646389
Model: SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Pearson's Coeffecient = 0.8935878127304003


In [ ]:
torch.cuda.empty_cache()